# Discriminant Analysis: sklearn, load_digits 
# by Vivian Zeng 09/15/2020

In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.metrics import roc_auc_score, precision_score, recall_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA, QuadraticDiscriminantAnalysis as QDA
from sklearn.svm import SVC

### 1. Load the the Heart Disease dataset. Print the first few rows. (5 pts)

In [3]:
heart = pd.read_csv('heart.csv')
heart.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,hd
0,63,male,typical_angina,145,233,higher_than_120,left_vent_hypertrophy,150,no,2.3,downsloping,0.0,fixed_defect,0
1,67,male,asymptomatic,160,286,lower_than_120,left_vent_hypertrophy,108,yes,1.5,flat,3.0,normal,1
2,67,male,asymptomatic,120,229,lower_than_120,left_vent_hypertrophy,129,yes,2.6,flat,2.0,reversable_defect,1
3,37,male,non_anginal_pain,130,250,lower_than_120,normal,187,no,3.5,downsloping,0.0,normal,0
4,41,female,atypical_angina,130,204,lower_than_120,left_vent_hypertrophy,172,no,1.4,upsloping,0.0,normal,0


### 2. Transform the data for modeling. (10 pts)

Create a data frame with all of the variables.

In [4]:
heart = heart.dropna()
heart = pd.get_dummies(heart, drop_first=True)
heart.head()

,age,trestbps,chol,thalach,oldpeak,ca,hd,sex_male,cp_atypical_angina,cp_non_anginal_pain,cp_typical_angina,fbs_lower_than_120,restecg_normal,restecg_stt_wave_abnormality,exang_yes,slope_flat,slope_upsloping,thal_normal,thal_reversable_defect
0,63,145,233,150,2.3,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0
1,67,160,286,108,1.5,3.0,1,1,0,0,0,1,0,0,1,1,0,1,0
2,67,120,229,129,2.6,2.0,1,1,0,0,0,1,0,0,1,1,0,0,1
3,37,130,250,187,3.5,0.0,0,1,0,1,0,1,1,0,0,0,0,1,0
4,41,130,204,172,1.4,0.0,0,0,1,0,0,1,0,0,0,0,1,1,0


### 3. Create training testing sets. (10 pts)

Create a feature matrix and response (target) vector for heart disease, and store these as numpy arrays.

In [5]:
y = heart.hd.values
X = heart.drop('hd', axis=1).values

Split the data into training and test sets using a 70%/30% split, stratifying on heart disease.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y, random_state=2019)
scaler = StandardScaler()
num_cols = list(range(6)) # standardize only numerical columns (OPTIONAL)
X_train[:, num_cols] = scaler.fit_transform(X_train[:, num_cols])
X_test[:, num_cols] = scaler.transform(X_test[:, num_cols])

Print the dimensions of the feature matrices and response vectors for both sets.

In [9]:
print('feature matrice training set dimensions:', X_train.shape)
print('feature matrice testing set dimensions:', X_test.shape)
print('response vector training set dimensions:', y_train.shape)
print('response vector testing set dimensions:', y_test.shape)

feature matrice training set dimensions: (207, 18)
feature matrice testing set dimensions: (90, 18)
response vector training set dimensions: (207,)
response vector testing set dimensions: (90,)


### 4.Fit a support vector classifier (SVM with a linear kernel) to the training data. Use cross validation to choose C based on the highest AUC ROC. Calculate recall, precision, and AUC ROC on both the training and test sets. (20 pts) Hint: try values of C between .01 and 10

In [10]:
def get_metrics(mod, X_train, X_test, y_train, y_test):
    """ Returns a data frame of metrics (precision,
        recall, AUC ROC) from training and test sets.
        Assumes model has decision_function() method.
        This will at least work for SVC, LDA, QDA.
    """
    pred_train = mod.predict(X_train)
    pred_test = mod.predict(X_test)
    score_train = mod.decision_function(X_train)
    score_test = mod.decision_function(X_test)
    # For LDA/QDA we could also use predict_proba,
    # but this works for both.
    recall_train = recall_score(y_train, pred_train)
    recall_test = recall_score(y_test, pred_test)
    precision_train = precision_score(y_train, pred_train)
    precision_test = precision_score(y_test, pred_test)
    rocauc_train = roc_auc_score(y_train, pred_train)
    rocauc_test = roc_auc_score(y_test, pred_test)
    metrics = {'Set':['Train', 'Test'],
               'Recall':[recall_train, recall_test],
               'Precision':[precision_train, precision_test],
               'ROC AUC':[rocauc_train, rocauc_test]}
    return pd.DataFrame(metrics)

In [11]:
# Values of C to try:
Cs = np.logspace(-2, 1)
print(Cs.round(3))

[ 0.01   0.012  0.013  0.015  0.018  0.02   0.023  0.027  0.031  0.036
  0.041  0.047  0.054  0.063  0.072  0.083  0.095  0.11   0.126  0.146
  0.168  0.193  0.222  0.256  0.295  0.339  0.391  0.45   0.518  0.596
  0.687  0.791  0.91   1.048  1.207  1.389  1.6    1.842  2.121  2.442
  2.812  3.237  3.728  4.292  4.942  5.69   6.551  7.543  8.685 10.   ]


In [12]:
svc = SVC(kernel = 'linear')
svc_cv = GridSearchCV(svc, param_grid={'C':Cs}, scoring='roc_auc', cv=5)
svc_cv.fit(X_train, y_train)
get_metrics(svc_cv, X_train, X_test, y_train, y_test)

,Set,Recall,Precision,ROC AUC
0,Train,0.810526,0.875000,0.856156
1,Test,0.785714,0.846154,0.830357


### 5. Fit an SVM model with radial basis kernel to the training data. Use cross validation to choose C based on the highest AUC ROC. Calculate recall, precision, and AUC ROC on both the training and test sets. (20 pts) Hint: try values of C between .01 and 10

In [13]:
svm = SVC(kernel = 'rbf', gamma='auto')
svm_cv = GridSearchCV(svm, param_grid={'C':Cs}, scoring='roc_auc', cv=5)
svm_cv.fit(X_train, y_train)
get_metrics(svm_cv, X_train, X_test, y_train, y_test)

,Set,Recall,Precision,ROC AUC
0,Train,0.926316,0.926316,0.931908
1,Test,0.785714,0.785714,0.799107


### 6. Fit a model using Linear Discriminant Analysis (LDA) to the training data. Calculate recall, precision, and AUC ROC on both the training and test sets. (15 pts)

In [14]:
lda = LDA().fit(X_train, y_train)
get_metrics(lda, X_train, X_test, y_train, y_test)

,Set,Recall,Precision,ROC AUC
0,Train,0.810526,0.875000,0.856156
1,Test,0.809524,0.971429,0.894345


### 7. Fit a model using Quadratic Discriminant Analysis (QDA) to the training data. Calculate recall, precision, and AUC ROC on both the training and test sets. (15 pts)

In [15]:
qda = QDA().fit(X_train, y_train)
get_metrics(qda, X_train, X_test, y_train, y_test)

,Set,Recall,Precision,ROC AUC
0,Train,0.810526,0.895349,0.865085
1,Test,0.809524,0.871795,0.852679


### 8. Write a few sentences comparing the performance of the models fit in questions (4) - (7). (5 pts)

Using ROC AUC to evaluate the models, LDA was the best (AUC=0.89), followed by QDA (AUC=0.85), SVC (AUC=0.83) then SVM
(AUC=0.80). This is somewhat surprising because LDA is such a simple model. The SVM seems to overfit to the training set because the
training AUC is the highest, but the test AUC is the lowest. The LDA model is tied for having the lowest training AUC, but because it doesn't
overfit the data, it still does well on the test set.